# 02 Prepare illustrative debt estimates

In this notebook, we prepare the illustrative carbon debt pathways that we will use for panel b of our figure.

In [1]:
import pyam
import pandas as pd
import numpy as np
from pathlib import Path

Step 1: Read in the data on regional emissions and debts.

In [2]:
r10 = pd.read_csv(
    Path(
        'data/r10_ndclts_impren_rcbyear.csv'
    )
)

Step 2: Cast the necessary columns to a `pyam.IamDataFrame`

In [3]:
r10.columns

Index(['model', 'case', 'r10', 'year', 'terr_GtCO2', 'terr_GtCO2_cmltv_1990',
       'terr_GtCO2_cmltv_2015', 'category', 'ppp_pf', 'rcb1990', 'rcb2015',
       'rcbyear', 'exceedanceyear', 'debtyear', 'exceedanceshareyear', 'pop'],
      dtype='object')

In [4]:
r10_for_panel_b = pyam.IamDataFrame(
    r10
    .rename(
        columns={
            'case':'scenario',
            'r10':'region'
        }
    )
     .groupby(
        ['model', 'scenario', 'year', 'region', 'category']
    )[['terr_GtCO2', 'terr_GtCO2_cmltv_1990', 'rcb1990']]
    .sum()
    .stack()
    .unstack(level='year')
    .reset_index()
    .rename(
        columns={
            'level_4':'variable'
        }
    )
    .assign(
        unit='Gt CO2'
    )
)

Step 3: Aggregate a 'World' region

In [5]:
r10_for_panel_b.aggregate_region(
    variable=r10_for_panel_b.variable,
    region='World',
    subregions=r10_for_panel_b.region,
    append=True
)

Step 4: Now, we want to calculate the share of remaining carbon budget consumed in each year.

In [6]:
r10_for_panel_b.divide(
    a='terr_GtCO2_cmltv_1990',
    b='rcb1990',
    ignore_units='unit',
    name='share_consumption_rcb',
    append=True
)

In [7]:
r10_for_panel_b.convert_unit(
    current='unit',
    to='%',
    factor=100,
    inplace=True
)

Step 5: Finally, filter out for the necessary regions, scenarios, categories, and variable and read this out for further processing.

In [8]:
r10_readout = r10_for_panel_b.filter(
    region=[
        'R10AFRICA',
        'R10EUROPE',
        'World'
    ],
    scenario=[
        'E',
        'IMP-REN'
    ],
    category='PP1990',
    variable='share_consumption_rcb'
)

In [9]:
r10_readout_ts = r10_readout.timeseries()

In [10]:
r10_readout_ts[1990] = 0

In [11]:
r10_readout = pyam.IamDataFrame(
    r10_readout_ts
)

In [12]:
r10_readout.to_excel(
    'data/02_data.xlsx'
)